In [1]:
# https://agentpy.readthedocs.io/en/latest/

# Model design
import agentpy as ap
import random
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import IPython

In [26]:
class settlementModel(ap.Model):

    def setup(self):

        # Create agents (houses)
        ## "house density" and "size" are defined parameters
        ## Square the length of grid
        ## Multiply by density to obtain number of agents
        n_houses = int(self.p['house density'] * (self.p.size**2))
        # AgentList function initializes the agents of desired number.
        houses = self.agents = ap.AgentList(self, n_houses)

        # Create grid (settlement)
        ## Grid composed of length^2 spots which is empty
        self.settlement = ap.Grid(self, [self.p.size]*2, track_empty=True)
        ## Add the agents into the house at random
        self.settlement.add_agents(houses, random=True, empty=True)
        
        # Create roads
        for k in range(2):
            origin = np.random.randint(self.p.size, size=2)
            print(origin)
            direction = np.random.rand()*np.pi
            print(direction)
            for i in range(100):
                road = np.trunc([origin[0] + i*np.cos(direction), origin[1] + i*np.sin(direction)])
                demo = self.settlement.grid[int(road[0])%100, int(road[1])%100]
                for m in demo:
                    for j in m:
                        d = self.settlement.neighbors(j,1).to_list()
                        self.settlement.remove_agents(d)
                    self.settlement.remove_agents(m)
                if i%7 == 0 and np.random.rand() < 0.5:
                    for q in range(-25,25):
                        subroad = np.trunc([road[0] + q*np.cos(direction + np.pi/2), road[1] + q*np.sin(direction + np.pi/2)])
                        demo = self.settlement.grid[int(subroad[0])%100, int(subroad[1])%100]
                        for m in demo:
                            self.settlement.remove_agents(m)

        # Initiate a dynamic variable for all houses
        # Condition 0: wooden, 1: concrete, 2: Burning, 3: Burned
        self.agents.condition = 0
        self.agents.burnstate = 0
        self.agents.material = "wooden"
        
        number_concrete = int(self.p['percentage_of_concrete']* n_houses)
        concrete_houses = self.agents.random(number_concrete)
        concrete_houses.condition = 1
        concrete_houses.material = "concrete"
        
        # Start a fire from the center of the grid
        ## 3 columns, all rows
        ## TODO change method of removing agents to not mess this up
        unfortunate_houses = self.agents.random(self.p.initial)
        unfortunate_houses.condition = 2

    def step(self):
        
        embers = self.agents.select(self.agents.condition == 5)
        for house in embers:
            house.condition = 2

        # Select burning houses
        burning_houses = self.agents.select(self.agents.condition == 2)

        # Spread fire
        for house in burning_houses:
            neighbors = self.settlement.neighbors(house).to_list()
            # Spread by direct contact
            for neighbor in neighbors:
                if neighbor.condition == 0 or neighbor.condition == 1:
                    burnProbability = self.p['probability-of-spread']
                    direction = list(np.subtract(self.settlement.positions[neighbor],self.settlement.positions[house]))
                    if direction == [-1,0]:
                        burnProbability -= self.p['south-wind-speed']
                    elif direction == [1,0]:
                        burnProbability += self.p['south-wind-speed']
                    elif direction == [0,-1]:
                        burnProbability -= self.p['east-wind-speed']
                    elif direction == [0,1]:
                        burnProbability += self.p['east-wind-speed']
                    else:
                        continue
                    if neighbor.material == "wooden":
                        if np.random.rand() < burnProbability:
                            neighbor.condition = 2
                    elif neighbor.material == "concrete":
                        if np.random.rand() < burnProbability/2:
                            neighbor.condition = 2
            # Spread by embers
            if self.p['big-jumps'] == True:
                ember = np.trunc([self.p['south-wind-speed']*5, self.p['east-wind-speed']*5])
                hloc = self.settlement.positions[house]
                floc = np.clip([int(hloc[0]+ember[0]), int(hloc[1]+ember[1])], 0, 99)
                farneighbor = self.settlement.grid[floc[0], floc[1]][0]
                for m in farneighbor:
                   # n = self.settlement.neighbors(m, 1)
                    #for x in n:
                    if np.random.rand() <= self.p['probability-of-spread'] and m.condition in [0,1]:
                        m.condition = 5
            
            # Burn out after n time steps burning
            house.burnstate += 1
            if house.material == "wooden":
                if house.burnstate >= 1:
                    house.condition = 3
            if house.material == "concrete":
                if house.burnstate >= 3:
                    house.condition = 3
                
        # Introduce new state 4: firefighting response
        burned_houses = len(self.agents.select(self.agents.condition == 3))
        if burned_houses / len(self.agents) >= self.p['firefighter-threshold']:
            if len(burning_houses) > 0:
                firefighter_location = burning_houses.random(1)
                for house in firefighter_location:
                    neighbor.condition = 4
        
        firefighting_houses = self.agents.select(self.agents.condition == 4)
        for house in firefighting_houses:
            neighbors = self.settlement.neighbors(house, 3)
            for neighbor in neighbors:
                if neighbor.condition == 2:
                    neighbor.condition = 4
        

        # Stop simulation if no fire is left
        if len(burning_houses) == 0:
            self.stop()

    def end(self):

        # Document a measure at the end of the simulation
        burned_houses = len(self.agents.select(self.agents.condition == 3))
        self.report('Percentage of burned houses',
                    burned_houses / len(self.agents))

In [27]:
# Define parameters

parameters = {
    'house density': 0.9, # Percentage of grid covered by houses
    'percentage_of_concrete': 0.3,
    'initial': 2,
    'size': 100, # Height and length of the grid
    'steps': 200,
    'probability-of-spread': 0.4,
    'south-wind-speed': np.random.rand()*2 - 1,
    'east-wind-speed': np.random.rand()*2 - 1,
    'firefighter-threshold': 0.1,
    'big-jumps': True
}

In [29]:
# Define parameters

parameters = {
    'house density': 0.9, # Percentage of grid covered by houses
    'percentage_of_concrete': 0.5,
    'initial': 2,
    'size': 100, # Height and length of the grid
    'steps': 200,
    'probability-of-spread': 0.5,
    'south-wind-speed': 0.8,
    'east-wind-speed': 0.8,
    'firefighter-threshold': 0.1,
    'big-jumps': True
}

# Create single-run animation with custom colors

def animation_plot(model, ax):
    attr_grid = model.settlement.attr_grid('condition')
    color_dict = {0:'#CDCDCD', 1: '#999999', 2:'#BF3E2D', 3:'#FFB159', 4:'#4248FF', 5: '#000000', None:'#d5e5d5'}
    ap.gridplot(attr_grid, ax=ax, color_dict=color_dict, convert=True)
    ax.set_title(f"Simulation of fire in informal settlements\n"
                 f"Time-step: {model.t}, Houses left: "
                 f"{len(model.agents.select(model.agents.condition == 0) + model.agents.select(model.agents.condition == 1))}")

fig, ax = plt.subplots(figsize = (8,8))
model = settlementModel(parameters)

animation = ap.animate(model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml(fps=15))

[38 77]
0.5983173483356892
[17 93]
2.9806630440586344


AgentIter (7 objects)
